# Lecture des données

Quand j'explore/analyse des données, la première chose que je fais est toujours :

In [1]:
import pandas as pd

Pour information/rappel,

In [2]:
pd.__version__

'0.17.1'

Pour lire un fichier CSV, nous utilisons la bien nommée fonction...

In [3]:
pd.read_csv('data/enfants.csv')

,prénom;nom;âge;garde
0,Toto;Le Magnifique;5;maman
1,Toto;Le Magnifique;5;papa
2,Charlie;Brown;99;99
3,Titi;Canary;4;papa et maman
4,Suzanne;Lancellotti;3;99
5,Toto;Le Zombie;4;maman


Ah, oui, j'ai pensé que votre feuille Excel serait exportée avec «;» comme caractère séparateur, car certains champs sont plus susceptibles de contenir des virgules...

In [4]:
pd.read_csv('data/enfants.csv', sep=';')

,prénom,nom,âge,garde
0,Toto,Le Magnifique,5,maman
1,Toto,Le Magnifique,5,papa
2,Charlie,Brown,99,99
3,Titi,Canary,4,papa et maman
4,Suzanne,Lancellotti,3,99
5,Toto,Le Zombie,4,maman


Bien. On dirait que les valeurs inconnues/manquantes sont indiquées par «99». Alors spécifions cela dans les options de la merveilleuse fonction `read_csv()`.

In [5]:
pd.read_csv('data/enfants.csv', sep=';', na_values=99)

,prénom,nom,âge,garde
0,Toto,Le Magnifique,5,maman
1,Toto,Le Magnifique,5,papa
2,Charlie,Brown,NaN,NaN
3,Titi,Canary,4,papa et maman
4,Suzanne,Lancellotti,3,NaN
5,Toto,Le Zombie,4,maman


Nous allons donner un nom à ce «data frame» (structure de données très pratique), ce sera `enfants`.

In [6]:
enfants = pd.read_csv('data/enfants.csv', sep=';', na_values='99')

Nous pouvons accéder aux données de garde avec la syntaxe (intuitive) suivante :

In [7]:
enfants['garde']

0            maman
1             papa
2              NaN
3    papa et maman
4              NaN
5            maman
Name: garde, dtype: object

ou bien `enfants.loc[:, 'garde']`.

# Retrait des doublons

Remarquons que nous avons deux entrées pour Toto Le Magnifique. Si nous ne voulons conserver qu'une entrée (ligne) par enfant, nous pouvons utiliser la méthode `drop_duplicates()`.

In [8]:
enfants.drop_duplicates(subset=['prénom', 'nom'])

,prénom,nom,âge,garde
0,Toto,Le Magnifique,5,maman
2,Charlie,Brown,NaN,NaN
3,Titi,Canary,4,papa et maman
4,Suzanne,Lancellotti,3,NaN
5,Toto,Le Zombie,4,maman


Par défaut, c'est la première entrée qui est conservée (voir la [documentation](http://pandas.pydata.org/pandas-docs/version/0.17.1/generated/pandas.DataFrame.drop_duplicates.html)). Nous perdons alors l'information contenue dans les autres entrées. Nous voudrions plutôt les grouper.

In [9]:
enfants.groupby(by=['prénom', 'nom'])['garde'].sum()

prénom   nom          
Charlie  Brown                      NaN
Suzanne  Lancellotti                NaN
Titi     Canary           papa et maman
Toto     Le Magnifique        mamanpapa
         Le Zombie                maman
Name: garde, dtype: object

Par défaut, `sum()` concatène. Pour une meilleure lisibilité, nous voulons peut-être appliquer une fonction faite maison.

In [10]:
enfants.groupby(by=['prénom', 'nom'])['garde'].apply(lambda x: '%s' % ', '.join(x.astype(str)))

prénom   nom          
Charlie  Brown                      nan
Suzanne  Lancellotti                nan
Titi     Canary           papa et maman
Toto     Le Magnifique      maman, papa
         Le Zombie                maman
Name: garde, dtype: object

Ecrivons une fonction (à appliquer au data frame).

In [11]:
def groupe_garde(x):
    return pd.Series(dict(age = x['âge'].mean(), garde_complete = '%s' % ', '.join(x['garde'].astype(str))))

In [12]:
enfants.groupby(by=['prénom', 'nom']).apply(groupe_garde)

age garde_complete
prénom  nom                              
Charlie Brown          NaN            nan
Suzanne Lancellotti      3            nan
Titi    Canary           4  papa et maman
Toto    Le Magnifique    5    maman, papa
        Le Zombie        4          maman

# Sortie du résultat

In [13]:
enfants.groupby(by=['prénom', 'nom']).apply(groupe_garde).to_csv('results/enfants_cleanup.csv', sep=';', na_rep='nan')

# Autres manipulations

Et si nous voulions continuer...

In [14]:
pd.read_csv('results/enfants_cleanup.csv', sep=';', na_values='nan')

,prénom,nom,age,garde_complete
0,Charlie,Brown,NaN,NaN
1,Suzanne,Lancellotti,3,NaN
2,Titi,Canary,4,papa et maman
3,Toto,Le Magnifique,5,"maman, papa"
4,Toto,Le Zombie,4,maman
